**Paso 1: Configuración inicial y carga de datos**

In [ ]:
# Configuración inicial
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Configurar estilo de gráficas
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Cargar datos - ajusta las rutas según tu estructura
archivo_referencia = '/content/drive/MyDrive/Colab Notebooks/TFM/referencia_analista.csv'
archivo_gpd = '/content/drive/MyDrive/Colab Notebooks/TFM/resultados_gpd.csv'

# Cargar datasets
print("Cargando datos...")
df_referencia = pd.read_csv(archivo_referencia)
df_gpd = pd.read_csv(archivo_gpd)

print(f"✓ Datos de referencia: {len(df_referencia)} eventos")
print(f"✓ Datos GPD: {len(df_gpd)} eventos")
print(f"✓ Columnas referencia: {list(df_referencia.columns)}")
print(f"✓ Columnas GPD: {list(df_gpd.columns)}")

**Paso 2: Exploración inicial de los datos**

In [ ]:
# Exploración básica de los datos de referencia
print("=== EXPLORACIÓN DATOS DE REFERENCIA ===")
print(f"Forma del dataset: {df_referencia.shape}")
print(f"Estaciones únicas: {sorted(df_referencia['Estacion'].unique())}")
print(f"Rango de fechas T-ini: {df_referencia['T-ini'].min()} a {df_referencia['T-ini'].max()}")

# Estadísticas básicas de SNR
print("\n=== ESTADÍSTICAS SNR ===")
print("SNR-P:")
print(df_referencia['SNR-P'].describe())
print("\nSNR-S:")
print(df_referencia['SNR-S'].describe())

# Verificar valores nulos en columnas clave
print("\n=== VALORES NULOS ===")
columnas_clave = ['T-P', 'T-S', 'SNR-P', 'SNR-S', 'Pond T-P', 'Pond T-S']
for col in columnas_clave:
    nulos = df_referencia[col].isnull().sum()
    print(f"{col}: {nulos} nulos ({nulos/len(df_referencia)*100:.1f}%)")

**Paso 3: Distribuciones de SNR**

In [ ]:
# Visualizar distribuciones de SNR
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Histograma SNR-P
axes[0,0].hist(df_referencia['SNR-P'].dropna(), bins=50, alpha=0.7, color='blue', edgecolor='black')
axes[0,0].set_xlabel('SNR-P (dB)')
axes[0,0].set_ylabel('Frecuencia')
axes[0,0].set_title('Distribución SNR Fase P')
axes[0,0].grid(True, alpha=0.3)

# Histograma SNR-S
axes[0,1].hist(df_referencia['SNR-S'].dropna(), bins=50, alpha=0.7, color='red', edgecolor='black')
axes[0,1].set_xlabel('SNR-S (dB)')
axes[0,1].set_ylabel('Frecuencia')
axes[0,1].set_title('Distribución SNR Fase S')
axes[0,1].grid(True, alpha=0.3)

# Box plot comparativo
data_box = [df_referencia['SNR-P'].dropna(), df_referencia['SNR-S'].dropna()]
axes[1,0].boxplot(data_box, labels=['SNR-P', 'SNR-S'])
axes[1,0].set_ylabel('SNR (dB)')
axes[1,0].set_title('Comparación SNR P vs S')
axes[1,0].grid(True, alpha=0.3)

# Scatter plot SNR-P vs SNR-S
valid_snr = df_referencia.dropna(subset=['SNR-P', 'SNR-S'])
axes[1,1].scatter(valid_snr['SNR-P'], valid_snr['SNR-S'], alpha=0.6, s=20)
axes[1,1].set_xlabel('SNR-P (dB)')
axes[1,1].set_ylabel('SNR-S (dB)')
axes[1,1].set_title('SNR-P vs SNR-S')
axes[1,1].grid(True, alpha=0.3)

# Línea de igualdad
min_snr = min(valid_snr['SNR-P'].min(), valid_snr['SNR-S'].min())
max_snr = max(valid_snr['SNR-P'].max(), valid_snr['SNR-S'].max())
axes[1,1].plot([min_snr, max_snr], [min_snr, max_snr], 'r--', alpha=0.7, label='SNR-P = SNR-S')
axes[1,1].legend()

plt.tight_layout()
plt.show()

# Estadísticas de correlación
correlacion = valid_snr['SNR-P'].corr(valid_snr['SNR-S'])
print(f"\nCorrelación SNR-P vs SNR-S: {correlacion:.3f}")

**Paso 4: Merge de datos y verificación**

In [ ]:
# Función para parsear timestamps (adaptada del script original)
def parse_timestamp(ts_str):
    """Convierte timestamp a datetime con manejo robusto de formatos"""
    if pd.isna(ts_str) or ts_str == 'NA':
        return None
    try:
        ts_str = str(ts_str).replace('Z', '+00:00')
        if '+' not in ts_str and 'T' in ts_str:
            ts_str += '+00:00'
        return pd.to_datetime(ts_str)
    except:
        return None

# Merge de datos
print("Realizando merge de datos...")
df_combined = pd.merge(
    df_referencia[['Estacion', 'mseed', 'T-P', 'T-S', 'Pond T-P', 'Pond T-S', 'SNR-P', 'SNR-S']],
    df_gpd[['Estacion', 'mseed', 'T-P', 'T-S', 'Pond T-P', 'Pond T-S']],
    on=['Estacion', 'mseed'],
    how='inner',
    suffixes=('_ref', '_gpd')
)

print(f"✓ Eventos coincidentes: {len(df_combined)}")
print(f"✓ Estaciones en datos combinados: {sorted(df_combined['Estacion'].unique())}")

# Verificar el merge
total_ref = len(df_referencia)
total_gpd = len(df_gpd)
total_merged = len(df_combined)

print(f"\nEficiencia del merge:")
print(f"- Eventos referencia utilizados: {total_merged}/{total_ref} ({total_merged/total_ref*100:.1f}%)")
print(f"- Eventos GPD utilizados: {total_merged}/{total_gpd} ({total_merged/total_gpd*100:.1f}%)")

# Mostrar primeras filas para verificación
print(f"\nPrimeras 3 filas del dataset combinado:")
print(df_combined[['Estacion', 'mseed', 'SNR-P', 'SNR-S']].head(3))

**Paso 5: Distribución de SNR por estación**

In [ ]:
# Análisis de distribución de SNR por estación
print("=== ANÁLISIS SNR POR ESTACIÓN ===")

estaciones = sorted(df_combined['Estacion'].unique())
print(f"Estaciones disponibles: {estaciones}")
print(f"Número de estaciones: {len(estaciones)}")

# Estadísticas básicas por estación
print("\n=== ESTADÍSTICAS DESCRIPTIVAS POR ESTACIÓN ===")
stats_por_estacion = df_combined.groupby('Estacion')[['SNR-P', 'SNR-S']].agg([
    'count', 'mean', 'median', 'std', 'min', 'max'
]).round(2)

print(stats_por_estacion)

In [ ]:
# Gráfica 1: Box plots por estación
fig, axes = plt.subplots(2, 1, figsize=(15, 12))

# Box plot SNR-P por estación
df_combined.boxplot(column='SNR-P', by='Estacion', ax=axes[0])
axes[0].set_title('Distribución SNR-P por Estación')
axes[0].set_xlabel('Estación')
axes[0].set_ylabel('SNR-P (dB)')
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# Box plot SNR-S por estación
df_combined.boxplot(column='SNR-S', by='Estacion', ax=axes[1])
axes[1].set_title('Distribución SNR-S por Estación')
axes[1].set_xlabel('Estación')
axes[1].set_ylabel('SNR-S (dB)')
axes[1].grid(True, alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Gráfica 2: Violin plots más detallados
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Violin plot SNR-P
sns.violinplot(data=df_combined, x='Estacion', y='SNR-P', ax=axes[0])
axes[0].set_title('Distribución Detallada SNR-P por Estación')
axes[0].set_xlabel('Estación')
axes[0].set_ylabel('SNR-P (dB)')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# Violin plot SNR-S
sns.violinplot(data=df_combined, x='Estacion', y='SNR-S', ax=axes[1])
axes[1].set_title('Distribución Detallada SNR-S por Estación')
axes[1].set_xlabel('Estación')
axes[1].set_ylabel('SNR-S (dB)')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Gráfica 3: Heatmap comparativo
# Crear matriz de estadísticas para visualización
stats_matrix_p = df_combined.groupby('Estacion')['SNR-P'].agg(['mean', 'median', 'std']).round(1)
stats_matrix_s = df_combined.groupby('Estacion')['SNR-S'].agg(['mean', 'median', 'std']).round(1)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Heatmap SNR-P
sns.heatmap(stats_matrix_p.T, annot=True, cmap='RdYlBu_r', ax=axes[0],
            cbar_kws={'label': 'SNR-P (dB)'})
axes[0].set_title('Estadísticas SNR-P por Estación')
axes[0].set_xlabel('Estación')
axes[0].set_ylabel('Estadística')

# Heatmap SNR-S
sns.heatmap(stats_matrix_s.T, annot=True, cmap='RdYlBu_r', ax=axes[1],
            cbar_kws={'label': 'SNR-S (dB)'})
axes[1].set_title('Estadísticas SNR-S por Estación')
axes[1].set_xlabel('Estación')
axes[1].set_ylabel('Estadística')

plt.tight_layout()
plt.show()

In [ ]:
# Análisis estadístico: Test ANOVA para diferencias significativas
from scipy import stats

print("\n=== ANÁLISIS ESTADÍSTICO ===")

# Preparar datos por estación para ANOVA
grupos_snr_p = [df_combined[df_combined['Estacion'] == est]['SNR-P'].dropna()
                for est in estaciones]
grupos_snr_s = [df_combined[df_combined['Estacion'] == est]['SNR-S'].dropna()
                for est in estaciones]

# Test ANOVA
f_stat_p, p_value_p = stats.f_oneway(*grupos_snr_p)
f_stat_s, p_value_s = stats.f_oneway(*grupos_snr_s)

print(f"ANOVA SNR-P entre estaciones:")
print(f"  F-statistic: {f_stat_p:.3f}")
print(f"  p-value: {p_value_p:.6f}")
print(f"  Diferencias significativas: {'SÍ' if p_value_p < 0.05 else 'NO'}")

print(f"\nANOVA SNR-S entre estaciones:")
print(f"  F-statistic: {f_stat_s:.3f}")
print(f"  p-value: {p_value_s:.6f}")
print(f"  Diferencias significativas: {'SÍ' if p_value_s < 0.05 else 'NO'}")

In [ ]:
# Gráfica 4: Scatter plot SNR-P vs SNR-S coloreado por estación
plt.figure(figsize=(12, 8))

colors = plt.cm.tab10(np.linspace(0, 1, len(estaciones)))
for i, estacion in enumerate(estaciones):
    data_est = df_combined[df_combined['Estacion'] == estacion]
    plt.scatter(data_est['SNR-P'], data_est['SNR-S'],
               label=estacion, alpha=0.7, s=30, color=colors[i])

plt.xlabel('SNR-P (dB)')
plt.ylabel('SNR-S (dB)')
plt.title('SNR-P vs SNR-S por Estación')
plt.grid(True, alpha=0.3)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Línea de igualdad
min_snr = min(df_combined['SNR-P'].min(), df_combined['SNR-S'].min())
max_snr = max(df_combined['SNR-P'].max(), df_combined['SNR-S'].max())
plt.plot([min_snr, max_snr], [min_snr, max_snr], 'k--', alpha=0.5, label='SNR-P = SNR-S')

plt.tight_layout()
plt.show()

In [ ]:
# Tabla resumen: Ranking de estaciones por SNR
print("\n=== RANKING DE ESTACIONES POR SNR ===")

ranking_data = []
for estacion in estaciones:
    data_est = df_combined[df_combined['Estacion'] == estacion]
    ranking_data.append({
        'Estacion': estacion,
        'N_eventos': len(data_est),
        'SNR-P_medio': data_est['SNR-P'].mean(),
        'SNR-S_medio': data_est['SNR-S'].mean(),
        'SNR-P_mediano': data_est['SNR-P'].median(),
        'SNR-S_mediano': data_est['SNR-S'].median(),
        'SNR-P_std': data_est['SNR-P'].std(),
        'SNR-S_std': data_est['SNR-S'].std()
    })

df_ranking = pd.DataFrame(ranking_data)

# Ordenar por SNR-P medio descendente
df_ranking_p = df_ranking.sort_values('SNR-P_medio', ascending=False)
print("Ranking por SNR-P medio:")
print(df_ranking_p[['Estacion', 'N_eventos', 'SNR-P_medio', 'SNR-P_mediano', 'SNR-P_std']].round(2))

print(f"\nRanking por SNR-S medio:")
df_ranking_s = df_ranking.sort_values('SNR-S_medio', ascending=False)
print(df_ranking_s[['Estacion', 'N_eventos', 'SNR-S_medio', 'SNR-S_mediano', 'SNR-S_std']].round(2))

**Paso 6: Identificación de estaciones problemáticas vs robustas**

In [ ]:
# Función para calcular métricas de robustez por estación
def calcular_metricas_robustez(df_combined):
    """Calcula métricas que indican robustez de cada estación"""

    metricas_estacion = []

    for estacion in sorted(df_combined['Estacion'].unique()):
        data_est = df_combined[df_combined['Estacion'] == estacion].copy()

        # Métricas básicas
        n_eventos = len(data_est)

        # Métricas de SNR
        snr_p_stats = {
            'snr_p_mean': data_est['SNR-P'].mean(),
            'snr_p_median': data_est['SNR-P'].median(),
            'snr_p_std': data_est['SNR-P'].std(),
            'snr_p_min': data_est['SNR-P'].min(),
            'snr_p_q25': data_est['SNR-P'].quantile(0.25),
            'snr_p_q75': data_est['SNR-P'].quantile(0.75)
        }

        snr_s_stats = {
            'snr_s_mean': data_est['SNR-S'].mean(),
            'snr_s_median': data_est['SNR-S'].median(),
            'snr_s_std': data_est['SNR-S'].std(),
            'snr_s_min': data_est['SNR-S'].min(),
            'snr_s_q25': data_est['SNR-S'].quantile(0.25),
            'snr_s_q75': data_est['SNR-S'].quantile(0.75)
        }

        # Métricas de robustez
        # 1. Porcentaje de eventos con SNR bajo
        eventos_snr_p_bajo = (data_est['SNR-P'] < 10).sum()  # <10 dB considerado bajo
        eventos_snr_s_bajo = (data_est['SNR-S'] < 10).sum()

        # 2. Consistencia (coeficiente de variación)
        cv_p = snr_p_stats['snr_p_std'] / snr_p_stats['snr_p_mean'] if snr_p_stats['snr_p_mean'] > 0 else np.inf
        cv_s = snr_s_stats['snr_s_std'] / snr_s_stats['snr_s_mean'] if snr_s_stats['snr_s_mean'] > 0 else np.inf

        # 3. Rango intercuartílico (IQR) - menor IQR = más consistente
        iqr_p = snr_p_stats['snr_p_q75'] - snr_p_stats['snr_p_q25']
        iqr_s = snr_s_stats['snr_s_q75'] - snr_s_stats['snr_s_q25']

        # 4. Score de robustez compuesto (0-100, mayor = más robusta)
        # Basado en: SNR medio, consistencia, y porcentaje de eventos de calidad
        score_snr_p = min(snr_p_stats['snr_p_mean'] / 20 * 40, 40)  # Max 40 puntos por SNR medio
        score_snr_s = min(snr_s_stats['snr_s_mean'] / 20 * 40, 40)  # Max 40 puntos por SNR medio

        score_consistencia_p = max(0, 20 - cv_p * 10)  # Max 20 puntos por consistencia
        score_consistencia_s = max(0, 20 - cv_s * 10)  # Max 20 puntos por consistencia

        score_calidad_p = (1 - eventos_snr_p_bajo/n_eventos) * 20  # Max 20 puntos por pocos eventos malos
        score_calidad_s = (1 - eventos_snr_s_bajo/n_eventos) * 20  # Max 20 puntos por pocos eventos malos

        robustez_score_p = score_snr_p + score_consistencia_p + score_calidad_p
        robustez_score_s = score_snr_s + score_consistencia_s + score_calidad_s
        robustez_score_total = (robustez_score_p + robustez_score_s) / 2

        metricas_estacion.append({
            'Estacion': estacion,
            'N_eventos': n_eventos,
            **snr_p_stats,
            **snr_s_stats,
            'eventos_snr_p_bajo': eventos_snr_p_bajo,
            'eventos_snr_s_bajo': eventos_snr_s_bajo,
            'pct_snr_p_bajo': eventos_snr_p_bajo/n_eventos*100,
            'pct_snr_s_bajo': eventos_snr_s_bajo/n_eventos*100,
            'cv_p': cv_p,
            'cv_s': cv_s,
            'iqr_p': iqr_p,
            'iqr_s': iqr_s,
            'robustez_score_p': robustez_score_p,
            'robustez_score_s': robustez_score_s,
            'robustez_score_total': robustez_score_total
        })

    return pd.DataFrame(metricas_estacion)

# Calcular métricas
print("=== ANÁLISIS DE ROBUSTEZ POR ESTACIÓN ===")
df_robustez = calcular_metricas_robustez(df_combined)

In [ ]:
# Clasificar estaciones por robustez
def clasificar_estaciones(df_robustez):
    """Clasifica estaciones en robustas, intermedias y problemáticas"""

    # Usar percentiles para clasificación
    q33 = df_robustez['robustez_score_total'].quantile(0.33)
    q67 = df_robustez['robustez_score_total'].quantile(0.67)

    def clasificar(score):
        if score >= q67:
            return 'Robusta'
        elif score >= q33:
            return 'Intermedia'
        else:
            return 'Problemática'

    df_robustez['Clasificacion'] = df_robustez['robustez_score_total'].apply(clasificar)
    return df_robustez, q33, q67

df_robustez, umbral_prob, umbral_rob = clasificar_estaciones(df_robustez)

# Mostrar clasificación
print(f"Umbrales de clasificación:")
print(f"  Problemática: Score < {umbral_prob:.1f}")
print(f"  Intermedia: {umbral_prob:.1f} ≤ Score < {umbral_rob:.1f}")
print(f"  Robusta: Score ≥ {umbral_rob:.1f}")

print(f"\n=== CLASIFICACIÓN DE ESTACIONES ===")
for categoria in ['Problemática', 'Intermedia', 'Robusta']:
    estaciones_cat = df_robustez[df_robustez['Clasificacion'] == categoria]['Estacion'].tolist()
    print(f"{categoria}: {estaciones_cat} ({len(estaciones_cat)} estaciones)")

In [ ]:
# Tabla resumen ordenada por robustez
print(f"\n=== RANKING COMPLETO DE ROBUSTEZ ===")
df_ranking_robustez = df_robustez.sort_values('robustez_score_total', ascending=False)

columnas_mostrar = ['Estacion', 'Clasificacion', 'N_eventos', 'robustez_score_total',
                   'snr_p_mean', 'snr_s_mean', 'pct_snr_p_bajo', 'pct_snr_s_bajo',
                   'cv_p', 'cv_s']

print(df_ranking_robustez[columnas_mostrar].round(2))

In [ ]:
# Visualización 1: Gráfica de robustez general
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Scores de robustez por estación
estaciones_ord = df_robustez.sort_values('robustez_score_total')['Estacion']
scores_ord = df_robustez.sort_values('robustez_score_total')['robustez_score_total']
colores = ['red' if score < umbral_prob else 'orange' if score < umbral_rob else 'green'
           for score in scores_ord]

axes[0,0].barh(range(len(estaciones_ord)), scores_ord, color=colores, alpha=0.7)
axes[0,0].set_yticks(range(len(estaciones_ord)))
axes[0,0].set_yticklabels(estaciones_ord)
axes[0,0].set_xlabel('Score de Robustez Total')
axes[0,0].set_title('Ranking de Robustez por Estación')
axes[0,0].axvline(umbral_prob, color='red', linestyle='--', alpha=0.7, label='Umbral Problemática')
axes[0,0].axvline(umbral_rob, color='green', linestyle='--', alpha=0.7, label='Umbral Robusta')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# 2. SNR medio vs Variabilidad
axes[0,1].scatter(df_robustez['snr_p_mean'], df_robustez['cv_p'],
                 c=[{'Robusta': 'green', 'Intermedia': 'orange', 'Problemática': 'red'}[cat]
                    for cat in df_robustez['Clasificacion']], alpha=0.7, s=80)
axes[0,1].set_xlabel('SNR-P Medio (dB)')
axes[0,1].set_ylabel('Coeficiente de Variación SNR-P')
axes[0,1].set_title('SNR Medio vs Variabilidad (Fase P)')
axes[0,1].grid(True, alpha=0.3)

# Añadir etiquetas de estación
for idx, row in df_robustez.iterrows():
    axes[0,1].annotate(row['Estacion'], (row['snr_p_mean'], row['cv_p']),
                      xytext=(5, 5), textcoords='offset points', fontsize=8)

# 3. Porcentaje de eventos con SNR bajo
x_pos = range(len(df_robustez))
axes[1,0].bar([x-0.2 for x in x_pos], df_robustez.sort_values('robustez_score_total')['pct_snr_p_bajo'],
              width=0.4, label='SNR-P < 10dB', alpha=0.7, color='blue')
axes[1,0].bar([x+0.2 for x in x_pos], df_robustez.sort_values('robustez_score_total')['pct_snr_s_bajo'],
              width=0.4, label='SNR-S < 10dB', alpha=0.7, color='red')
axes[1,0].set_xticks(x_pos)
axes[1,0].set_xticklabels(df_robustez.sort_values('robustez_score_total')['Estacion'], rotation=45)
axes[1,0].set_ylabel('Porcentaje de Eventos (%)')
axes[1,0].set_title('Eventos con SNR Bajo por Estación')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# 4. Comparación SNR-P vs SNR-S por clasificación
for categoria, color in [('Robusta', 'green'), ('Intermedia', 'orange'), ('Problemática', 'red')]:
    data_cat = df_robustez[df_robustez['Clasificacion'] == categoria]
    axes[1,1].scatter(data_cat['snr_p_mean'], data_cat['snr_s_mean'],
                     label=categoria, color=color, alpha=0.7, s=80)

axes[1,1].set_xlabel('SNR-P Medio (dB)')
axes[1,1].set_ylabel('SNR-S Medio (dB)')
axes[1,1].set_title('SNR-P vs SNR-S por Clasificación')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

# Línea de igualdad
min_snr = min(df_robustez['snr_p_mean'].min(), df_robustez['snr_s_mean'].min())
max_snr = max(df_robustez['snr_p_mean'].max(), df_robustez['snr_s_mean'].max())
axes[1,1].plot([min_snr, max_snr], [min_snr, max_snr], 'k--', alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# Análisis detallado de estaciones problemáticas
print(f"\n=== ANÁLISIS DETALLADO DE ESTACIONES PROBLEMÁTICAS ===")

estaciones_problematicas = df_robustez[df_robustez['Clasificacion'] == 'Problemática']['Estacion'].tolist()
estaciones_robustas = df_robustez[df_robustez['Clasificacion'] == 'Robusta']['Estacion'].tolist()

if estaciones_problematicas:
    print(f"\nEstaciones PROBLEMÁTICAS ({len(estaciones_problematicas)}):")
    for estacion in estaciones_problematicas:
        data_est = df_robustez[df_robustez['Estacion'] == estacion].iloc[0]
        print(f"\n{estacion}:")
        print(f"  • Score robustez: {data_est['robustez_score_total']:.1f}/100")
        print(f"  • SNR-P medio: {data_est['snr_p_mean']:.1f} dB (std: {data_est['snr_p_std']:.1f})")
        print(f"  • SNR-S medio: {data_est['snr_s_mean']:.1f} dB (std: {data_est['snr_s_std']:.1f})")
        print(f"  • Eventos con SNR-P bajo: {data_est['pct_snr_p_bajo']:.1f}%")
        print(f"  • Eventos con SNR-S bajo: {data_est['pct_snr_s_bajo']:.1f}%")
        print(f"  • Variabilidad P (CV): {data_est['cv_p']:.2f}")
        print(f"  • Variabilidad S (CV): {data_est['cv_s']:.2f}")

        # Identificar problemas principales
        problemas = []
        if data_est['snr_p_mean'] < 15: problemas.append("SNR-P bajo")
        if data_est['snr_s_mean'] < 15: problemas.append("SNR-S bajo")
        if data_est['pct_snr_p_bajo'] > 30: problemas.append("Muchos eventos P con SNR<10")
        if data_est['pct_snr_s_bajo'] > 30: problemas.append("Muchos eventos S con SNR<10")
        if data_est['cv_p'] > 0.5: problemas.append("Alta variabilidad P")
        if data_est['cv_s'] > 0.5: problemas.append("Alta variabilidad S")

        if problemas:
            print(f"  • Problemas principales: {', '.join(problemas)}")

print(f"\nEstaciones ROBUSTAS ({len(estaciones_robustas)}) para comparación:")
for estacion in estaciones_robustas:
    data_est = df_robustez[df_robustez['Estacion'] == estacion].iloc[0]
    print(f"  {estacion}: Score {data_est['robustez_score_total']:.1f}, "
          f"SNR-P {data_est['snr_p_mean']:.1f}±{data_est['snr_p_std']:.1f}, "
          f"SNR-S {data_est['snr_s_mean']:.1f}±{data_est['snr_s_std']:.1f}")

In [ ]:
# Recomendaciones basadas en el análisis
print(f"\n=== RECOMENDACIONES ===")

# Estadísticas generales
total_estaciones = len(df_robustez)
pct_problematicas = len(estaciones_problematicas) / total_estaciones * 100
pct_robustas = len(estaciones_robustas) / total_estaciones * 100

print(f"📊 Resumen general:")
print(f"  • {pct_problematicas:.1f}% de estaciones son problemáticas")
print(f"  • {pct_robustas:.1f}% de estaciones son robustas")

if estaciones_problematicas:
    print(f"\n🔧 Recomendaciones para estaciones problemáticas:")
    print(f"  • Revisar calibración de instrumentos")
    print(f"  • Verificar condiciones ambientales (ruido sísmico)")
    print(f"  • Considerar filtrado adicional en preprocesamiento")
    print(f"  • Ajustar parámetros del modelo GPD específicamente para estas estaciones")
    print(f"  • Implementar umbrales adaptativos por estación")

print(f"\n📈 Para el análisis del modelo GPD:")
print(f"  • Esperar mayor dificultad en estaciones: {estaciones_problematicas}")
print(f"  • Usar estaciones robustas como benchmark: {estaciones_robustas[:3]}")
print(f"  • Considerar pesos diferentes por estación en evaluación")

# Guardar resultados para análisis posteriores
df_robustez.to_csv('/content/analisis_robustez_estaciones.csv', index=False)
print(f"\n💾 Análisis de robustez guardado en: /content/analisis_robustez_estaciones.csv")

**Paso 7: Impacto de estaciones problemáticas en métricas del modelo GPD**

In [ ]:
# Primero necesitamos procesar los timestamps y calcular métricas del modelo GPD
print("=== PROCESANDO DATOS PARA ANÁLISIS DEL MODELO GPD ===")

# Función para parsear timestamps (del script original)
def parse_timestamp(ts_str):
    """Convierte timestamp a datetime con manejo robusto de formatos"""
    if pd.isna(ts_str) or ts_str == 'NA':
        return None
    try:
        ts_str = str(ts_str).replace('Z', '+00:00')
        if '+' not in ts_str and 'T' in ts_str:
            ts_str += '+00:00'
        return pd.to_datetime(ts_str)
    except:
        return None

# Parsear timestamps
print("Procesando timestamps...")
df_combined['T-P_ref_dt'] = df_combined['T-P_ref'].apply(parse_timestamp)
df_combined['T-S_ref_dt'] = df_combined['T-S_ref'].apply(parse_timestamp)
df_combined['T-P_gpd_dt'] = df_combined['T-P_gpd'].apply(parse_timestamp)
df_combined['T-S_gpd_dt'] = df_combined['T-S_gpd'].apply(parse_timestamp)

# Calcular errores temporales
df_combined['error_temporal_P'] = df_combined.apply(
    lambda row: (row['T-P_gpd_dt'] - row['T-P_ref_dt']).total_seconds()
    if pd.notna(row['T-P_gpd_dt']) and pd.notna(row['T-P_ref_dt']) else None, axis=1
)

df_combined['error_temporal_S'] = df_combined.apply(
    lambda row: (row['T-S_gpd_dt'] - row['T-S_ref_dt']).total_seconds()
    if pd.notna(row['T-S_gpd_dt']) and pd.notna(row['T-S_ref_dt']) else None, axis=1
)

print(f"✓ Errores temporales calculados")
print(f"✓ Detecciones P válidas: {df_combined['error_temporal_P'].notna().sum()}")
print(f"✓ Detecciones S válidas: {df_combined['error_temporal_S'].notna().sum()}")

In [ ]:
# Función para calcular métricas del modelo por estación
def calcular_metricas_modelo_por_estacion(df, tolerancia=1.0):
    """Calcula métricas de detección del modelo GPD por estación"""

    metricas_modelo = []

    for estacion in sorted(df['Estacion'].unique()):
        data_est = df[df['Estacion'] == estacion].copy()
        n_eventos = len(data_est)

        # Tasas de detección
        detecciones_p = data_est['T-P_gpd_dt'].notna().sum()
        detecciones_s = data_est['T-S_gpd_dt'].notna().sum()
        detecciones_ambas = ((data_est['T-P_gpd_dt'].notna()) & (data_est['T-S_gpd_dt'].notna())).sum()

        tasa_deteccion_p = detecciones_p / n_eventos * 100
        tasa_deteccion_s = detecciones_s / n_eventos * 100
        tasa_deteccion_ambas = detecciones_ambas / n_eventos * 100

        # Métricas de precisión temporal
        errores_p = data_est['error_temporal_P'].dropna()
        errores_s = data_est['error_temporal_S'].dropna()

        # Estadísticas de error P
        if len(errores_p) > 0:
            error_p_mean = errores_p.mean()
            error_p_median = errores_p.median()
            error_p_std = errores_p.std()
            error_p_rms = np.sqrt((errores_p ** 2).mean())
            error_p_mae = errores_p.abs().mean()

            # Clasificación de calidad P
            errores_p_abs = errores_p.abs()
            excelente_p = (errores_p_abs <= 1.0).sum()
            buena_p = ((errores_p_abs > 1.0) & (errores_p_abs <= 5.0)).sum()
            pobre_p = (errores_p_abs > 5.0).sum()

            pct_excelente_p = excelente_p / len(errores_p) * 100
            pct_buena_p = buena_p / len(errores_p) * 100
            pct_pobre_p = pobre_p / len(errores_p) * 100
        else:
            error_p_mean = error_p_median = error_p_std = error_p_rms = error_p_mae = np.nan
            pct_excelente_p = pct_buena_p = pct_pobre_p = 0

        # Estadísticas de error S
        if len(errores_s) > 0:
            error_s_mean = errores_s.mean()
            error_s_median = errores_s.median()
            error_s_std = errores_s.std()
            error_s_rms = np.sqrt((errores_s ** 2).mean())
            error_s_mae = errores_s.abs().mean()

            # Clasificación de calidad S
            errores_s_abs = errores_s.abs()
            excelente_s = (errores_s_abs <= 1.0).sum()
            buena_s = ((errores_s_abs > 1.0) & (errores_s_abs <= 5.0)).sum()
            pobre_s = (errores_s_abs > 5.0).sum()

            pct_excelente_s = excelente_s / len(errores_s) * 100
            pct_buena_s = buena_s / len(errores_s) * 100
            pct_pobre_s = pobre_s / len(errores_s) * 100
        else:
            error_s_mean = error_s_median = error_s_std = error_s_rms = error_s_mae = np.nan
            pct_excelente_s = pct_buena_s = pct_pobre_s = 0

        # Calcular métricas P/R/F1 simplificadas para esta tolerancia
        # Para fase P
        ref_p = data_est['T-P_ref_dt'].notna()
        det_p = data_est['T-P_gpd_dt'].notna()
        errores_p_todos = data_est['error_temporal_P']

        tp_p = ((ref_p & det_p) & (errores_p_todos.abs() <= tolerancia)).sum()
        fp_p = ((~ref_p & det_p) | ((ref_p & det_p) & (errores_p_todos.abs() > tolerancia))).sum()
        fn_p = ((ref_p & ~det_p) | ((ref_p & det_p) & (errores_p_todos.abs() > tolerancia))).sum()

        precision_p = tp_p / (tp_p + fp_p) if (tp_p + fp_p) > 0 else 0
        recall_p = tp_p / (tp_p + fn_p) if (tp_p + fn_p) > 0 else 0
        f1_p = (2 * precision_p * recall_p) / (precision_p + recall_p) if (precision_p + recall_p) > 0 else 0

        # Para fase S
        ref_s = data_est['T-S_ref_dt'].notna()
        det_s = data_est['T-S_gpd_dt'].notna()
        errores_s_todos = data_est['error_temporal_S']

        tp_s = ((ref_s & det_s) & (errores_s_todos.abs() <= tolerancia)).sum()
        fp_s = ((~ref_s & det_s) | ((ref_s & det_s) & (errores_s_todos.abs() > tolerancia))).sum()
        fn_s = ((ref_s & ~det_s) | ((ref_s & det_s) & (errores_s_todos.abs() > tolerancia))).sum()

        precision_s = tp_s / (tp_s + fp_s) if (tp_s + fp_s) > 0 else 0
        recall_s = tp_s / (tp_s + fn_s) if (tp_s + fn_s) > 0 else 0
        f1_s = (2 * precision_s * recall_s) / (precision_s + recall_s) if (precision_s + recall_s) > 0 else 0

        metricas_modelo.append({
            'Estacion': estacion,
            'N_eventos': n_eventos,
            'detecciones_p': detecciones_p,
            'detecciones_s': detecciones_s,
            'detecciones_ambas': detecciones_ambas,
            'tasa_deteccion_p': tasa_deteccion_p,
            'tasa_deteccion_s': tasa_deteccion_s,
            'tasa_deteccion_ambas': tasa_deteccion_ambas,
            'error_p_mean': error_p_mean,
            'error_p_median': error_p_median,
            'error_p_rms': error_p_rms,
            'error_p_mae': error_p_mae,
            'error_s_mean': error_s_mean,
            'error_s_median': error_s_median,
            'error_s_rms': error_s_rms,
            'error_s_mae': error_s_mae,
            'pct_excelente_p': pct_excelente_p,
            'pct_buena_p': pct_buena_p,
            'pct_pobre_p': pct_pobre_p,
            'pct_excelente_s': pct_excelente_s,
            'pct_buena_s': pct_buena_s,
            'pct_pobre_s': pct_pobre_s,
            'precision_p': precision_p,
            'recall_p': recall_p,
            'f1_p': f1_p,
            'precision_s': precision_s,
            'recall_s': recall_s,
            'f1_s': f1_s,
            'f1_promedio': (f1_p + f1_s) / 2
        })

    return pd.DataFrame(metricas_modelo)

# Calcular métricas del modelo
df_metricas_modelo = calcular_metricas_modelo_por_estacion(df_combined, tolerancia=1.0)
print("✓ Métricas del modelo GPD calculadas por estación")

In [ ]:
# Combinar con clasificación de robustez
df_analisis_completo = pd.merge(
    df_metricas_modelo,
    df_robustez[['Estacion', 'Clasificacion', 'robustez_score_total', 'snr_p_mean', 'snr_s_mean']],
    on='Estacion'
)

print("=== IMPACTO DE ESTACIONES PROBLEMÁTICAS EN MÉTRICAS DEL MODELO ===")
print(f"Total estaciones analizadas: {len(df_analisis_completo)}")

# Estadísticas por tipo de estación
for categoria in ['Problemática', 'Intermedia', 'Robusta']:
    data_cat = df_analisis_completo[df_analisis_completo['Clasificacion'] == categoria]
    if len(data_cat) > 0:
        print(f"\n{categoria.upper()} ({len(data_cat)} estaciones):")
        print(f"  Tasa detección P: {data_cat['tasa_deteccion_p'].mean():.1f}% (±{data_cat['tasa_deteccion_p'].std():.1f}%)")
        print(f"  Tasa detección S: {data_cat['tasa_deteccion_s'].mean():.1f}% (±{data_cat['tasa_deteccion_s'].std():.1f}%)")
        print(f"  F1-Score P: {data_cat['f1_p'].mean():.3f} (±{data_cat['f1_p'].std():.3f})")
        print(f"  F1-Score S: {data_cat['f1_s'].mean():.3f} (±{data_cat['f1_s'].std():.3f})")
        print(f"  Error RMS P: {data_cat['error_p_rms'].mean():.2f}s (±{data_cat['error_p_rms'].std():.2f}s)")
        print(f"  Error RMS S: {data_cat['error_s_rms'].mean():.2f}s (±{data_cat['error_s_rms'].std():.2f}s)")

In [ ]:
# Visualización del impacto
fig, axes = plt.subplots(3, 2, figsize=(16, 18))

# Colores por categoría
colores_cat = {'Robusta': 'green', 'Intermedia': 'orange', 'Problemática': 'red'}

# 1. Tasas de detección por categoría
categorias = ['Problemática', 'Intermedia', 'Robusta']
tasas_p = [df_analisis_completo[df_analisis_completo['Clasificacion'] == cat]['tasa_deteccion_p'].mean()
           for cat in categorias]
tasas_s = [df_analisis_completo[df_analisis_completo['Clasificacion'] == cat]['tasa_deteccion_s'].mean()
           for cat in categorias]

x_pos = np.arange(len(categorias))
width = 0.35

axes[0,0].bar(x_pos - width/2, tasas_p, width, label='Fase P', alpha=0.7, color='blue')
axes[0,0].bar(x_pos + width/2, tasas_s, width, label='Fase S', alpha=0.7, color='red')
axes[0,0].set_xlabel('Clasificación de Estación')
axes[0,0].set_ylabel('Tasa de Detección (%)')
axes[0,0].set_title('Tasas de Detección por Tipo de Estación')
axes[0,0].set_xticks(x_pos)
axes[0,0].set_xticklabels(categorias)
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# 2. F1-Scores por categoría
f1_p = [df_analisis_completo[df_analisis_completo['Clasificacion'] == cat]['f1_p'].mean()
        for cat in categorias]
f1_s = [df_analisis_completo[df_analisis_completo['Clasificacion'] == cat]['f1_s'].mean()
        for cat in categorias]

axes[0,1].bar(x_pos - width/2, f1_p, width, label='F1 Fase P', alpha=0.7, color='blue')
axes[0,1].bar(x_pos + width/2, f1_s, width, label='F1 Fase S', alpha=0.7, color='red')
axes[0,1].set_xlabel('Clasificación de Estación')
axes[0,1].set_ylabel('F1-Score')
axes[0,1].set_title('F1-Scores por Tipo de Estación')
axes[0,1].set_xticks(x_pos)
axes[0,1].set_xticklabels(categorias)
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# 3. Scatter: SNR vs F1-Score P
for categoria, color in colores_cat.items():
    data_cat = df_analisis_completo[df_analisis_completo['Clasificacion'] == categoria]
    axes[1,0].scatter(data_cat['snr_p_mean'], data_cat['f1_p'],
                     label=categoria, color=color, alpha=0.7, s=80)

axes[1,0].set_xlabel('SNR-P Medio (dB)')
axes[1,0].set_ylabel('F1-Score Fase P')
axes[1,0].set_title('SNR-P vs F1-Score Fase P')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Añadir línea de tendencia
from scipy.stats import pearsonr
x_vals = df_analisis_completo['snr_p_mean'].dropna()
y_vals = df_analisis_completo['f1_p'].dropna()
if len(x_vals) > 1:
    z = np.polyfit(x_vals, y_vals, 1)
    p = np.poly1d(z)
    axes[1,0].plot(x_vals, p(x_vals), "r--", alpha=0.8)
    corr, _ = pearsonr(x_vals, y_vals)
    axes[1,0].text(0.05, 0.95, f'r = {corr:.3f}', transform=axes[1,0].transAxes,
                  bbox=dict(boxstyle="round", facecolor='white', alpha=0.8))

# 4. Scatter: SNR vs F1-Score S
for categoria, color in colores_cat.items():
    data_cat = df_analisis_completo[df_analisis_completo['Clasificacion'] == categoria]
    axes[1,1].scatter(data_cat['snr_s_mean'], data_cat['f1_s'],
                     label=categoria, color=color, alpha=0.7, s=80)

axes[1,1].set_xlabel('SNR-S Medio (dB)')
axes[1,1].set_ylabel('F1-Score Fase S')
axes[1,1].set_title('SNR-S vs F1-Score Fase S')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

# Línea de tendencia para S
x_vals_s = df_analisis_completo['snr_s_mean'].dropna()
y_vals_s = df_analisis_completo['f1_s'].dropna()
if len(x_vals_s) > 1:
    z_s = np.polyfit(x_vals_s, y_vals_s, 1)
    p_s = np.poly1d(z_s)
    axes[1,1].plot(x_vals_s, p_s(x_vals_s), "r--", alpha=0.8)
    corr_s, _ = pearsonr(x_vals_s, y_vals_s)
    axes[1,1].text(0.05, 0.95, f'r = {corr_s:.3f}', transform=axes[1,1].transAxes,
                  bbox=dict(boxstyle="round", facecolor='white', alpha=0.8))

# 5. Errores RMS por categoría
error_rms_p = [df_analisis_completo[df_analisis_completo['Clasificacion'] == cat]['error_p_rms'].mean()
               for cat in categorias]
error_rms_s = [df_analisis_completo[df_analisis_completo['Clasificacion'] == cat]['error_s_rms'].mean()
               for cat in categorias]

axes[2,0].bar(x_pos - width/2, error_rms_p, width, label='Error RMS P', alpha=0.7, color='blue')
axes[2,0].bar(x_pos + width/2, error_rms_s, width, label='Error RMS S', alpha=0.7, color='red')
axes[2,0].set_xlabel('Clasificación de Estación')
axes[2,0].set_ylabel('Error RMS (segundos)')
axes[2,0].set_title('Errores RMS por Tipo de Estación')
axes[2,0].set_xticks(x_pos)
axes[2,0].set_xticklabels(categorias)
axes[2,0].legend()
axes[2,0].grid(True, alpha=0.3)

# 6. Distribución de calidad de detecciones
calidad_data = []
for categoria in categorias:
    data_cat = df_analisis_completo[df_analisis_completo['Clasificacion'] == categoria]
    pct_exc_p = data_cat['pct_excelente_p'].mean()
    pct_bue_p = data_cat['pct_buena_p'].mean()
    pct_pob_p = data_cat['pct_pobre_p'].mean()

    calidad_data.append([pct_exc_p, pct_bue_p, pct_pob_p])

calidad_data = np.array(calidad_data)
bottom_buena = calidad_data[:, 0]
bottom_pobre = calidad_data[:, 0] + calidad_data[:, 1]

axes[2,1].bar(x_pos, calidad_data[:, 0], label='Excelente (≤1s)', color='green', alpha=0.7)
axes[2,1].bar(x_pos, calidad_data[:, 1], bottom=bottom_buena, label='Buena (1-5s)', color='orange', alpha=0.7)
axes[2,1].bar(x_pos, calidad_data[:, 2], bottom=bottom_pobre, label='Pobre (>5s)', color='red', alpha=0.7)

axes[2,1].set_xlabel('Clasificación de Estación')
axes[2,1].set_ylabel('Porcentaje de Detecciones (%)')
axes[2,1].set_title('Calidad de Detecciones P por Tipo de Estación')
axes[2,1].set_xticks(x_pos)
axes[2,1].set_xticklabels(categorias)
axes[2,1].legend()
axes[2,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Análisis estadístico de diferencias
from scipy.stats import mannwhitneyu, kruskal

print(f"\n=== ANÁLISIS ESTADÍSTICO DE DIFERENCIAS ===")

# Test de Kruskal-Wallis para múltiples grupos
metricas_test = ['tasa_deteccion_p', 'tasa_deteccion_s', 'f1_p', 'f1_s', 'error_p_rms', 'error_s_rms']

for metrica in metricas_test:
    grupos = [df_analisis_completo[df_analisis_completo['Clasificacion'] == cat][metrica].dropna()
              for cat in categorias]

    # Filtrar grupos vacíos
    grupos = [g for g in grupos if len(g) > 0]

    if len(grupos) >= 2:
        h_stat, p_value = kruskal(*grupos)
        print(f"\n{metrica}:")
        print(f"  Kruskal-Wallis H: {h_stat:.3f}")
        print(f"  p-value: {p_value:.6f}")
        print(f"  Diferencias significativas: {'SÍ' if p_value < 0.05 else 'NO'}")

# Comparaciones par a par más importantes
print(f"\n=== COMPARACIONES PROBLEMÁTICAS vs ROBUSTAS ===")

prob_data = df_analisis_completo[df_analisis_completo['Clasificacion'] == 'Problemática']
rob_data = df_analisis_completo[df_analisis_completo['Clasificacion'] == 'Robusta']

if len(prob_data) > 0 and len(rob_data) > 0:
    comparaciones = [
        ('Tasa detección P', 'tasa_deteccion_p'),
        ('Tasa detección S', 'tasa_deteccion_s'),
        ('F1-Score P', 'f1_p'),
        ('F1-Score S', 'f1_s'),
        ('Error RMS P', 'error_p_rms'),
        ('Error RMS S', 'error_s_rms')
    ]

    for nombre, metrica in comparaciones:
        prob_vals = prob_data[metrica].dropna()
        rob_vals = rob_data[metrica].dropna()

        if len(prob_vals) > 0 and len(rob_vals) > 0:
            u_stat, p_value = mannwhitneyu(prob_vals, rob_vals, alternative='two-sided')

            prob_mean = prob_vals.mean()
            rob_mean = rob_vals.mean()
            diferencia = prob_mean - rob_mean
            pct_diferencia = (diferencia / rob_mean * 100) if rob_mean != 0 else 0

            print(f"\n{nombre}:")
            print(f"  Problemáticas: {prob_mean:.3f}")
            print(f"  Robustas: {rob_mean:.3f}")
            print(f"  Diferencia: {diferencia:+.3f} ({pct_diferencia:+.1f}%)")
            print(f"  p-value: {p_value:.6f} ({'significativo' if p_value < 0.05 else 'no significativo'})")

In [ ]:
# Tabla resumen final
print(f"\n=== RESUMEN CUANTITATIVO DEL IMPACTO ===")

# Crear tabla resumen
resumen_impacto = df_analisis_completo.groupby('Clasificacion').agg({
    'tasa_deteccion_p': ['mean', 'std', 'count'],
    'tasa_deteccion_s': ['mean', 'std'],
    'f1_p': ['mean', 'std'],
    'f1_s': ['mean', 'std'],
    'error_p_rms': ['mean', 'std'],
    'error_s_rms': ['mean', 'std'],
    'snr_p_mean': ['mean', 'std'],
    'snr_s_mean': ['mean', 'std']
}).round(3)

print("Tabla resumen por tipo de estación:")
print(resumen_impacto)

# Conclusiones del análisis
print(f"\n=== CONCLUSIONES DEL ANÁLISIS ===")

if len(prob_data) > 0 and len(rob_data) > 0:
    # Calcular impactos relativos
    impacto_f1_p = ((rob_data['f1_p'].mean() - prob_data['f1_p'].mean()) / prob_data['f1_p'].mean() * 100)
    impacto_f1_s = ((rob_data['f1_s'].mean() - prob_data['f1_s'].mean()) / prob_data['f1_s'].mean() * 100)
    impacto_tasa_p = ((rob_data['tasa_deteccion_p'].mean() - prob_data['tasa_deteccion_p'].mean()) / prob_data['tasa_deteccion_p'].mean() * 100)
    impacto_tasa_s = ((rob_data['tasa_deteccion_s'].mean() - prob_data['tasa_deteccion_s'].mean()) / prob_data['tasa_deteccion_s'].mean() * 100)

    print(f"✓ HIPÓTESIS CONFIRMADA: El SNR bajo impacta significativamente el rendimiento del modelo GPD")
    print(f"\n📊 Impactos cuantificados:")
    print(f"  • F1-Score P: {impacto_f1_p:+.1f}% mejor en estaciones robustas")
    print(f"  • F1-Score S: {impacto_f1_s:+.1f}% mejor en estaciones robustas")
    print(f"  • Tasa detección P: {impacto_tasa_p:+.1f}% mejor en estaciones robustas")
    print(f"  • Tasa detección S: {impacto_tasa_s:+.1f}% mejor en estaciones robustas")

    print(f"\n🎯 Implicaciones para el modelo:")
    print(f"  • SNR es un predictor fuerte del rendimiento del modelo GPD")
    print(f"  • Estaciones problemáticas requieren tratamiento especial")
    print(f"  • Los umbrales de detección podrían ajustarse por estación")
    print(f"  • El filtrado previo es más crítico en estaciones con SNR bajo")

# Guardar análisis completo
df_analisis_completo.to_csv('/content/analisis_completo_impacto_modelo.csv', index=False)
print(f"\n💾 Análisis completo guardado en: /content/analisis_completo_impacto_modelo.csv")

**Paso 8: Análisis detallado SNR vs Métricas del Modelo GPD**

In [ ]:
# Crear bins de SNR para análisis granular
print("=== ANÁLISIS GRANULAR: SNR vs MÉTRICAS DEL MODELO ===")

def crear_bins_snr(df, columna_snr, metodo='percentiles'):
    """Crea bins de SNR usando diferentes métodos"""

    if metodo == 'percentiles':
        # Bins basados en percentiles
        bins = [df[columna_snr].quantile(q) for q in [0, 0.2, 0.4, 0.6, 0.8, 1.0]]
        labels = ['Muy Bajo\n(0-20%)', 'Bajo\n(20-40%)', 'Medio\n(40-60%)',
                 'Alto\n(60-80%)', 'Muy Alto\n(80-100%)']
    elif metodo == 'fijo':
        # Bins fijos basados en conocimiento sísmico
        bins = [-np.inf, 5, 10, 15, 20, np.inf]
        labels = ['<5 dB', '5-10 dB', '10-15 dB', '15-20 dB', '>20 dB']
    else:
        # Bins adaptativos
        q25, q50, q75 = df[columna_snr].quantile([0.25, 0.5, 0.75])
        bins = [-np.inf, q25, q50, q75, np.inf]
        labels = [f'<{q25:.1f}', f'{q25:.1f}-{q50:.1f}',
                 f'{q50:.1f}-{q75:.1f}', f'>{q75:.1f}']

    return bins, labels

# Preparar datos para análisis por evento individual
df_eventos = df_combined.copy()

# Calcular métricas de detección por evento
df_eventos['detectado_p'] = df_eventos['T-P_gpd_dt'].notna()
df_eventos['detectado_s'] = df_eventos['T-S_gpd_dt'].notna()
df_eventos['detectado_ambas'] = df_eventos['detectado_p'] & df_eventos['detectado_s']

# Clasificar errores temporales
df_eventos['error_p_abs'] = df_eventos['error_temporal_P'].abs()
df_eventos['error_s_abs'] = df_eventos['error_temporal_S'].abs()

# Detección exitosa con diferentes tolerancias
tolerancias = [1.0, 2.0, 5.0]
for tol in tolerancias:
    df_eventos[f'exitosa_p_{tol}s'] = (df_eventos['detectado_p'] &
                                       (df_eventos['error_p_abs'] <= tol))
    df_eventos[f'exitosa_s_{tol}s'] = (df_eventos['detectado_s'] &
                                       (df_eventos['error_s_abs'] <= tol))

print(f"✓ Datos preparados: {len(df_eventos)} eventos")
print(f"✓ Rango SNR-P: {df_eventos['SNR-P'].min():.1f} a {df_eventos['SNR-P'].max():.1f} dB")
print(f"✓ Rango SNR-S: {df_eventos['SNR-S'].min():.1f} a {df_eventos['SNR-S'].max():.1f} dB")

In [ ]:
# Función para calcular métricas por bins de SNR
def calcular_metricas_por_bins_snr(df, columna_snr, fase, tolerancia=1.0, metodo_bins='fijo'):
    """Calcula métricas de detección por bins de SNR"""

    bins, labels = crear_bins_snr(df, columna_snr, metodo_bins)
    df_temp = df.dropna(subset=[columna_snr]).copy()
    df_temp['bin_snr'] = pd.cut(df_temp[columna_snr], bins=bins, labels=labels, include_lowest=True)

    metricas_bins = []

    for bin_label in labels:
        data_bin = df_temp[df_temp['bin_snr'] == bin_label]
        if len(data_bin) == 0:
            continue

        n_eventos = len(data_bin)

        # Preparar columnas según la fase
        if fase == 'P':
            ref_col = 'T-P_ref_dt'
            det_col = 'T-P_gpd_dt'
            error_col = 'error_temporal_P'
        else:  # fase == 'S'
            ref_col = 'T-S_ref_dt'
            det_col = 'T-S_gpd_dt'
            error_col = 'error_temporal_S'

        # Eventos de referencia (ground truth)
        eventos_ref = data_bin[ref_col].notna().sum()

        # Detecciones del modelo
        eventos_det = data_bin[det_col].notna().sum()

        # Cálculo de TP, FP, FN
        mask_ref = data_bin[ref_col].notna()
        mask_det = data_bin[det_col].notna()
        mask_both = mask_ref & mask_det

        # True Positives: detecciones correctas dentro de tolerancia
        errores_abs = data_bin[error_col].abs()
        tp = (mask_both & (errores_abs <= tolerancia)).sum()

        # False Negatives: referencias no detectadas o detectadas fuera de tolerancia
        fn = (mask_ref & ~mask_det).sum() + (mask_both & (errores_abs > tolerancia)).sum()

        # False Positives: detecciones sin referencia o fuera de tolerancia
        fp = (~mask_ref & mask_det).sum() + (mask_both & (errores_abs > tolerancia)).sum()

        # Métricas principales
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        # Tasa de detección simple
        tasa_deteccion = eventos_det / eventos_ref * 100 if eventos_ref > 0 else 0

        # Estadísticas de error (solo para detecciones válidas)
        errores_validos = data_bin[data_bin[det_col].notna()][error_col].dropna()
        if len(errores_validos) > 0:
            error_mean = errores_validos.mean()
            error_median = errores_validos.median()
            error_rms = np.sqrt((errores_validos ** 2).mean())
            error_std = errores_validos.std()
            error_mae = errores_validos.abs().mean()
        else:
            error_mean = error_median = error_rms = error_std = error_mae = np.nan

        # SNR estadísticas del bin
        snr_mean = data_bin[columna_snr].mean()
        snr_median = data_bin[columna_snr].median()
        snr_std = data_bin[columna_snr].std()

        metricas_bins.append({
            'bin_snr': bin_label,
            'snr_mean': snr_mean,
            'snr_median': snr_median,
            'snr_std': snr_std,
            'n_eventos': n_eventos,
            'eventos_ref': eventos_ref,
            'eventos_det': eventos_det,
            'tp': tp,
            'fp': fp,
            'fn': fn,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'tasa_deteccion': tasa_deteccion,
            'error_mean': error_mean,
            'error_median': error_median,
            'error_rms': error_rms,
            'error_std': error_std,
            'error_mae': error_mae
        })

    return pd.DataFrame(metricas_bins)

# Calcular métricas por bins para ambas fases
print("Calculando métricas por bins de SNR...")

df_bins_p = calcular_metricas_por_bins_snr(df_eventos, 'SNR-P', 'P', tolerancia=1.0)
df_bins_s = calcular_metricas_por_bins_snr(df_eventos, 'SNR-S', 'S', tolerancia=1.0)

print("✓ Métricas por bins calculadas")
print(f"✓ Bins SNR-P: {len(df_bins_p)}")
print(f"✓ Bins SNR-S: {len(df_bins_s)}")

In [ ]:
# Visualización 1: Métricas principales vs SNR
fig, axes = plt.subplots(2, 4, figsize=(20, 12))

# Función para plotear métricas por bins
def plot_metricas_bins(df_bins, fase, axes_row):
    x_pos = range(len(df_bins))
    labels = df_bins['bin_snr'].tolist()

    # F1-Score
    axes_row[0].bar(x_pos, df_bins['f1'], alpha=0.7, color='purple')
    axes_row[0].set_title(f'F1-Score vs SNR-{fase}')
    axes_row[0].set_ylabel('F1-Score')
    axes_row[0].set_xticks(x_pos)
    axes_row[0].set_xticklabels(labels, rotation=45)
    axes_row[0].grid(True, alpha=0.3)

    # Recall
    axes_row[1].bar(x_pos, df_bins['recall'], alpha=0.7, color='blue')
    axes_row[1].set_title(f'Recall vs SNR-{fase}')
    axes_row[1].set_ylabel('Recall')
    axes_row[1].set_xticks(x_pos)
    axes_row[1].set_xticklabels(labels, rotation=45)
    axes_row[1].grid(True, alpha=0.3)

    # Precision
    axes_row[2].bar(x_pos, df_bins['precision'], alpha=0.7, color='green')
    axes_row[2].set_title(f'Precision vs SNR-{fase}')
    axes_row[2].set_ylabel('Precision')
    axes_row[2].set_xticks(x_pos)
    axes_row[2].set_xticklabels(labels, rotation=45)
    axes_row[2].grid(True, alpha=0.3)

    # Error RMS
    axes_row[3].bar(x_pos, df_bins['error_rms'], alpha=0.7, color='red')
    axes_row[3].set_title(f'Error RMS vs SNR-{fase}')
    axes_row[3].set_ylabel('Error RMS (s)')
    axes_row[3].set_xticks(x_pos)
    axes_row[3].set_xticklabels(labels, rotation=45)
    axes_row[3].grid(True, alpha=0.3)

# Plotear para ambas fases
plot_metricas_bins(df_bins_p, 'P', axes[0])
plot_metricas_bins(df_bins_s, 'S', axes[1])

plt.tight_layout()
plt.show()

# Mostrar tablas numéricas
print("\n=== MÉTRICAS POR BINS SNR-P ===")
columnas_mostrar = ['bin_snr', 'snr_mean', 'n_eventos', 'f1', 'recall', 'precision', 'error_rms', 'error_median']
print(df_bins_p[columnas_mostrar].round(3))

print("\n=== MÉTRICAS POR BINS SNR-S ===")
print(df_bins_s[columnas_mostrar].round(3))

In [ ]:
# Visualización 2: Análisis de umbrales críticos
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Análisis continuo usando ventanas deslizantes
def analisis_continuo_snr(df, columna_snr, fase, ventana=50):
    """Análisis continuo de métricas vs SNR usando ventanas deslizantes"""

    df_sorted = df.dropna(subset=[columna_snr]).sort_values(columna_snr)

    if fase == 'P':
        det_col = 'detectado_p'
        error_col = 'error_temporal_P'
    else:
        det_col = 'detectado_s'
        error_col = 'error_temporal_S'

    snr_values = []
    recall_values = []
    precision_values = []
    f1_values = []
    error_rms_values = []
    error_median_values = []

    for i in range(len(df_sorted) - ventana + 1):
        ventana_data = df_sorted.iloc[i:i+ventana]

        snr_medio = ventana_data[columna_snr].mean()

        # Calcular recall (tasa de detección en esta ventana)
        recall = ventana_data[det_col].mean()

        # Para precision y F1, necesitamos más lógica (simplificado aquí)
        detecciones = ventana_data[det_col].sum()
        errores_valid = ventana_data[ventana_data[det_col]][error_col].dropna()

        if len(errores_valid) > 0:
            # Precision aproximada (detecciones "buenas" / total detecciones)
            precision_aprox = (errores_valid.abs() <= 1.0).mean()
            f1_aprox = 2 * recall * precision_aprox / (recall + precision_aprox) if (recall + precision_aprox) > 0 else 0

            error_rms = np.sqrt((errores_valid ** 2).mean())
            error_median = errores_valid.median()
        else:
            precision_aprox = 0
            f1_aprox = 0
            error_rms = np.nan
            error_median = np.nan

        snr_values.append(snr_medio)
        recall_values.append(recall)
        precision_values.append(precision_aprox)
        f1_values.append(f1_aprox)
        error_rms_values.append(error_rms)
        error_median_values.append(error_median)

    return (np.array(snr_values), np.array(recall_values), np.array(precision_values),
            np.array(f1_values), np.array(error_rms_values), np.array(error_median_values))

# Análisis continuo para ambas fases
if len(df_eventos) >= 100:  # Solo si hay suficientes datos
    ventana = min(100, len(df_eventos) // 10)

    snr_p, recall_p, prec_p, f1_p, rms_p, med_p = analisis_continuo_snr(df_eventos, 'SNR-P', 'P', ventana)
    snr_s, recall_s, prec_s, f1_s, rms_s, med_s = analisis_continuo_snr(df_eventos, 'SNR-S', 'S', ventana)

    # Plot análisis continuo
    # Fila 1: Fase P
    axes[0,0].plot(snr_p, f1_p, 'b-', alpha=0.7, linewidth=2)
    axes[0,0].set_xlabel('SNR-P (dB)')
    axes[0,0].set_ylabel('F1-Score')
    axes[0,0].set_title('F1-Score vs SNR-P (Continuo)')
    axes[0,0].grid(True, alpha=0.3)

    axes[0,1].plot(snr_p, recall_p, 'g-', alpha=0.7, linewidth=2)
    axes[0,1].set_xlabel('SNR-P (dB)')
    axes[0,1].set_ylabel('Recall')
    axes[0,1].set_title('Recall vs SNR-P (Continuo)')
    axes[0,1].grid(True, alpha=0.3)

    axes[0,2].plot(snr_p, rms_p, 'r-', alpha=0.7, linewidth=2)
    axes[0,2].set_xlabel('SNR-P (dB)')
    axes[0,2].set_ylabel('Error RMS (s)')
    axes[0,2].set_title('Error RMS vs SNR-P (Continuo)')
    axes[0,2].grid(True, alpha=0.3)

    # Fila 2: Fase S
    axes[1,0].plot(snr_s, f1_s, 'b-', alpha=0.7, linewidth=2)
    axes[1,0].set_xlabel('SNR-S (dB)')
    axes[1,0].set_ylabel('F1-Score')
    axes[1,0].set_title('F1-Score vs SNR-S (Continuo)')
    axes[1,0].grid(True, alpha=0.3)

    axes[1,1].plot(snr_s, recall_s, 'g-', alpha=0.7, linewidth=2)
    axes[1,1].set_xlabel('SNR-S (dB)')
    axes[1,1].set_ylabel('Recall')
    axes[1,1].set_title('Recall vs SNR-S (Continuo)')
    axes[1,1].grid(True, alpha=0.3)

    axes[1,2].plot(snr_s, rms_s, 'r-', alpha=0.7, linewidth=2)
    axes[1,2].set_xlabel('SNR-S (dB)')
    axes[1,2].set_ylabel('Error RMS (s)')
    axes[1,2].set_title('Error RMS vs SNR-S (Continuo)')
    axes[1,2].grid(True, alpha=0.3)

else:
    # Si no hay suficientes datos, usar scatter plots simples
    for i in range(2):
        for j in range(3):
            axes[i,j].text(0.5, 0.5, 'Datos insuficientes\npara análisis continuo',
                          ha='center', va='center', transform=axes[i,j].transAxes)
            axes[i,j].set_xticks([])
            axes[i,j].set_yticks([])

plt.tight_layout()
plt.show()

In [ ]:
# Análisis de umbrales críticos
print("\n=== ANÁLISIS DE UMBRALES CRÍTICOS ===")

def encontrar_umbrales_criticos(df_bins, metrica='recall', umbral=0.8):
    """Encuentra el SNR mínimo para mantener una métrica por encima del umbral"""

    # Ordenar por SNR medio
    df_sorted = df_bins.sort_values('snr_mean')

    # Encontrar primer bin que supera el umbral
    bins_sobre_umbral = df_sorted[df_sorted[metrica] >= umbral]

    if len(bins_sobre_umbral) > 0:
        snr_critico = bins_sobre_umbral.iloc[0]['snr_mean']
        return snr_critico, bins_sobre_umbral.iloc[0]
    else:
        return None, None

# Umbrales para diferentes métricas
umbrales_analisis = [
    ('Recall ≥ 80%', 'recall', 0.8),
    ('Precision ≥ 70%', 'precision', 0.7),
    ('F1-Score ≥ 70%', 'f1', 0.7),
    ('Error RMS ≤ 2s', 'error_rms', 2.0)  # Para esta métrica, buscamos por debajo
]

print("FASE P:")
for nombre, metrica, umbral in umbrales_analisis:
    if metrica == 'error_rms':
        # Para error RMS, buscamos el último bin por debajo del umbral
        df_sorted = df_bins_p.sort_values('snr_mean')
        bins_bajo_umbral = df_sorted[df_sorted[metrica] <= umbral]
        if len(bins_bajo_umbral) > 0:
            snr_critico = bins_bajo_umbral.iloc[-1]['snr_mean']
            print(f"  {nombre}: SNR-P ≥ {snr_critico:.1f} dB")
        else:
            print(f"  {nombre}: No se alcanza el umbral")
    else:
        snr_critico, bin_info = encontrar_umbrales_criticos(df_bins_p, metrica, umbral)
        if snr_critico:
            print(f"  {nombre}: SNR-P ≥ {snr_critico:.1f} dB")
        else:
            print(f"  {nombre}: No se alcanza el umbral")

print("\nFASE S:")
for nombre, metrica, umbral in umbrales_analisis:
    if metrica == 'error_rms':
        df_sorted = df_bins_s.sort_values('snr_mean')
        bins_bajo_umbral = df_sorted[df_sorted[metrica] <= umbral]
        if len(bins_bajo_umbral) > 0:
            snr_critico = bins_bajo_umbral.iloc[-1]['snr_mean']
            print(f"  {nombre}: SNR-S ≥ {snr_critico:.1f} dB")
        else:
            print(f"  {nombre}: No se alcanza el umbral")
    else:
        snr_critico, bin_info = encontrar_umbrales_criticos(df_bins_s, metrica, umbral)
        if snr_critico:
            print(f"  {nombre}: SNR-S ≥ {snr_critico:.1f} dB")
        else:
            print(f"  {nombre}: No se alcanza el umbral")

In [ ]:
# Análisis de correlaciones
print("\n=== ANÁLISIS DE CORRELACIONES ===")

from scipy.stats import pearsonr, spearmanr

# Preparar datos para correlaciones
datos_correlacion = df_eventos.dropna(subset=['SNR-P', 'SNR-S']).copy()

# Calcular correlaciones para eventos con detecciones válidas
correlaciones = []

metricas_corr = [
    ('SNR-P', 'detectado_p', 'Detección P'),
    ('SNR-P', 'error_p_abs', 'Error absoluto P'),
    ('SNR-S', 'detectado_s', 'Detección S'),
    ('SNR-S', 'error_s_abs', 'Error absoluto S')
]

for snr_col, metrica_col, nombre in metricas_corr:
    if metrica_col in datos_correlacion.columns:
        datos_validos = datos_correlacion.dropna(subset=[snr_col, metrica_col])

        if len(datos_validos) > 10:
            corr_pearson, p_pearson = pearsonr(datos_validos[snr_col], datos_validos[metrica_col])
            corr_spearman, p_spearman = spearmanr(datos_validos[snr_col], datos_validos[metrica_col])

            correlaciones.append({
                'Relación': f'{snr_col} vs {nombre}',
                'N': len(datos_validos),
                'Pearson_r': corr_pearson,
                'Pearson_p': p_pearson,
                'Spearman_rho': corr_spearman,
                'Spearman_p': p_spearman,
                'Significativa': 'Sí' if min(p_pearson, p_spearman) < 0.05 else 'No'
            })

df_correlaciones = pd.DataFrame(correlaciones)
print("Correlaciones SNR vs Métricas del Modelo:")
print(df_correlaciones.round(4))

In [ ]:
# Análisis de sesgos sistemáticos por rangos de SNR
print("\n=== ANÁLISIS DE SESGOS SISTEMÁTICOS ===")

def analizar_sesgos_por_snr(df, columna_snr, fase):
    """Analiza sesgos sistemáticos en diferentes rangos de SNR"""

    if fase == 'P':
        error_col = 'error_temporal_P'
    else:
        error_col = 'error_temporal_S'

    # Crear bins cuartiles
    q25, q50, q75 = df[columna_snr].quantile([0.25, 0.5, 0.75])
    bins = [-np.inf, q25, q50, q75, np.inf]
    labels = [f'Q1 (<{q25:.1f})', f'Q2 ({q25:.1f}-{q50:.1f})',
             f'Q3 ({q50:.1f}-{q75:.1f})', f'Q4 (>{q75:.1f})']

    df_temp = df.dropna(subset=[columna_snr, error_col]).copy()
    df_temp['cuartil_snr'] = pd.cut(df_temp[columna_snr], bins=bins, labels=labels)

    sesgos = []
    for cuartil in labels:
        data_cuartil = df_temp[df_temp['cuartil_snr'] == cuartil][error_col]

        if len(data_cuartil) > 5:
            sesgo_medio = data_cuartil.mean()
            sesgo_mediano = data_cuartil.median()
            desv_std = data_cuartil.std()
            n_eventos = len(data_cuartil)

            # Test de sesgo (¿es significativamente diferente de 0?)
            from scipy.stats import ttest_1samp
            t_stat, p_value = ttest_1samp(data_cuartil, 0)

            sesgos.append({
                'Cuartil': cuartil,
                'N': n_eventos,
                'Sesgo_medio': sesgo_medio,
                'Sesgo_mediano': sesgo_mediano,
                'Desv_std': desv_std,
                'T_stat': t_stat,
                'P_value': p_value,
                'Sesgo_significativo': 'Sí' if p_value < 0.05 else 'No'
            })

    return pd.DataFrame(sesgos)

# Analizar sesgos para ambas fases
print("SESGOS TEMPORALES POR CUARTILES DE SNR-P:")
sesgos_p = analizar_sesgos_por_snr(df_eventos, 'SNR-P', 'P')
print(sesgos_p.round(3))

print("\nSESGOS TEMPORALES POR CUARTILES DE SNR-S:")
sesgos_s = analizar_sesgos_por_snr(df_eventos, 'SNR-S', 'S')
print(sesgos_s.round(3))

# Guardar todos los resultados
print(f"\n💾 Guardando resultados del análisis...")
df_bins_p.to_csv('/content/analisis_snr_bins_fase_P.csv', index=False)
df_bins_s.to_csv('/content/analisis_snr_bins_fase_S.csv', index=False)
df_correlaciones.to_csv('/content/correlaciones_snr_metricas.csv', index=False)
sesgos_p.to_csv('/content/sesgos_temporales_snr_P.csv', index=False)
sesgos_s.to_csv('/content/sesgos_temporales_snr_S.csv', index=False)

print("✓ Análisis completo guardado en múltiples archivos CSV")

# Nueva sección

In [ ]:
# Resumen ejecutivo y conclusiones
print("\n" + "="*80)
print("RESUMEN EJECUTIVO: RELACIÓN SNR vs MÉTRICAS DEL MODELO GPD")
print("="*80)

# Encontrar tendencias principales
if len(df_bins_p) > 1:
    # Tendencia F1-Score
    f1_min_p = df_bins_p['f1'].min()
    f1_max_p = df_bins_p['f1'].max()
    mejora_f1_p = ((f1_max_p - f1_min_p) / f1_min_p * 100) if f1_min_p > 0 else 0

    # Tendencia Recall
    recall_min_p = df_bins_p['recall'].min()
    recall_max_p = df_bins_p['recall'].max()
    mejora_recall_p = ((recall_max_p - recall_min_p) / recall_min_p * 100) if recall_min_p > 0 else 0

    print(f"📊 HALLAZGOS PRINCIPALES FASE P:")
    print(f"  • F1-Score varía de {f1_min_p:.3f} a {f1_max_p:.3f} ({mejora_f1_p:+.1f}% mejora)")
    print(f"  • Recall varía de {recall_min_p:.3f} a {recall_max_p:.3f} ({mejora_recall_p:+.1f}% mejora)")

    # Bin con mejor rendimiento
    mejor_bin_p = df_bins_p.loc[df_bins_p['f1'].idxmax()]
    print(f"  • Mejor rendimiento en bin: {mejor_bin_p['bin_snr']} (SNR ~{mejor_bin_p['snr_mean']:.1f} dB)")

# Continuar con el resumen ejecutivo
if len(df_bins_s) > 1:
    f1_min_s = df_bins_s['f1'].min()
    f1_max_s = df_bins_s['f1'].max()
    mejora_f1_s = ((f1_max_s - f1_min_s) / f1_min_s * 100) if f1_min_s > 0 else 0

    recall_min_s = df_bins_s['recall'].min()
    recall_max_s = df_bins_s['recall'].max()
    mejora_recall_s = ((recall_max_s - recall_min_s) / recall_min_s * 100) if recall_min_s > 0 else 0

    print(f"\n📊 HALLAZGOS PRINCIPALES FASE S:")
    print(f"  • F1-Score varía de {f1_min_s:.3f} a {f1_max_s:.3f} ({mejora_f1_s:+.1f}% mejora)")
    print(f"  • Recall varía de {recall_min_s:.3f} a {recall_max_s:.3f} ({mejora_recall_s:+.1f}% mejora)")

    mejor_bin_s = df_bins_s.loc[df_bins_s['f1'].idxmax()]
    print(f"  • Mejor rendimiento en bin: {mejor_bin_s['bin_snr']} (SNR ~{mejor_bin_s['snr_mean']:.1f} dB)")

# Análisis de degradación crítica
print(f"\n🔍 ANÁLISIS DE DEGRADACIÓN CRÍTICA:")

# Encontrar puntos de quiebre en recall
def encontrar_punto_quiebre(df_bins, metrica='recall', umbral_critico=0.5):
    """Encuentra el punto donde la métrica cae drásticamente"""
    df_sorted = df_bins.sort_values('snr_mean')

    for i in range(len(df_sorted)-1):
        actual = df_sorted.iloc[i][metrica]
        siguiente = df_sorted.iloc[i+1][metrica]

        if actual < umbral_critico and siguiente >= umbral_critico:
            return df_sorted.iloc[i+1]['snr_mean'], df_sorted.iloc[i+1]

    return None, None

# Análisis para ambas fases
for fase, df_bins in [('P', df_bins_p), ('S', df_bins_s)]:
    if len(df_bins) > 1:
        # Punto donde recall cae por debajo del 50%
        snr_critico_50, bin_info = encontrar_punto_quiebre(df_bins, 'recall', 0.5)
        if snr_critico_50:
            print(f"  • Fase {fase}: Recall >50% requiere SNR ≥ {snr_critico_50:.1f} dB")

        # Punto donde F1 cae por debajo del 50%
        snr_critico_f1, _ = encontrar_punto_quiebre(df_bins, 'f1', 0.5)
        if snr_critico_f1:
            print(f"  • Fase {fase}: F1-Score >50% requiere SNR ≥ {snr_critico_f1:.1f} dB")

# Análisis de precisión vs recall trade-off
print(f"\n⚖️ ANÁLISIS PRECISION vs RECALL:")

for fase, df_bins in [('P', df_bins_p), ('S', df_bins_s)]:
    if len(df_bins) > 1:
        # Correlación entre precision y recall por bins
        if len(df_bins) >= 3:
            corr_pr = df_bins['precision'].corr(df_bins['recall'])
            print(f"  • Fase {fase}: Correlación Precision-Recall = {corr_pr:.3f}")

        # Identificar bins con alta precision pero bajo recall (conservador)
        conservadores = df_bins[(df_bins['precision'] > 0.7) & (df_bins['recall'] < 0.5)]
        if len(conservadores) > 0:
            print(f"  • Fase {fase}: {len(conservadores)} bins muestran comportamiento conservador (alta precision, bajo recall)")

        # Identificar bins con bajo precision pero alto recall (liberal)
        liberales = df_bins[(df_bins['precision'] < 0.5) & (df_bins['recall'] > 0.7)]
        if len(liberales) > 0:
            print(f"  • Fase {fase}: {len(liberales)} bins muestran comportamiento liberal (bajo precision, alto recall)")

print(f"\n🎯 RECOMENDACIONES TÉCNICAS:")

# Recomendaciones basadas en hallazgos
recomendaciones = []

# Recomendaciones de SNR mínimo
if len(df_bins_p) > 1:
    bins_buenos_p = df_bins_p[df_bins_p['f1'] > 0.6]
    if len(bins_buenos_p) > 0:
        snr_min_recomendado_p = bins_buenos_p['snr_mean'].min()
        recomendaciones.append(f"SNR-P mínimo recomendado: {snr_min_recomendado_p:.1f} dB para F1>0.6")

if len(df_bins_s) > 1:
    bins_buenos_s = df_bins_s[df_bins_s['f1'] > 0.6]
    if len(bins_buenos_s) > 0:
        snr_min_recomendado_s = bins_buenos_s['snr_mean'].min()
        recomendaciones.append(f"SNR-S mínimo recomendado: {snr_min_recomendado_s:.1f} dB para F1>0.6")

# Recomendaciones de preprocesamiento
if len(df_correlaciones) > 0:
    corr_significativas = df_correlaciones[df_correlaciones['Significativa'] == 'Sí']
    if len(corr_significativas) > 0:
        recomendaciones.append("Implementar filtrado adaptativo basado en SNR estimado en tiempo real")
        recomendaciones.append("Considerar umbrales de detección variables según SNR de la estación")

# Recomendaciones de calidad de datos
problemas_snr_bajo = df_eventos[df_eventos['SNR-P'] < 10]['Estacion'].value_counts()
if len(problemas_snr_bajo) > 0:
    estacion_mas_problematica = problemas_snr_bajo.index[0]
    eventos_problematicos = problemas_snr_bajo.iloc[0]
    recomendaciones.append(f"Revisar estación {estacion_mas_problematica} ({eventos_problematicos} eventos con SNR-P<10)")

# Imprimir recomendaciones
for i, rec in enumerate(recomendaciones, 1):
    print(f"  {i}. {rec}")

# Recomendaciones para mejora del modelo
print(f"\n🔧 ESTRATEGIAS PARA MEJORAR EL MODELO GPD:")
estrategias = [
    "Implementar umbrales adaptativos por estación basados en SNR histórico",
    "Desarrollar módulo de estimación de SNR en tiempo real",
    "Crear modelos especializados para diferentes rangos de SNR",
    "Implementar post-procesamiento para corregir sesgos temporales en SNR bajo",
    "Considerar ensemble de modelos optimizados para diferentes condiciones de SNR",
    "Desarrollar métricas de confianza basadas en SNR para filtrar detecciones inciertas"
]

for i, estrategia in enumerate(estrategias, 1):
    print(f"  {i}. {estrategia}")

In [ ]:
# Visualización final: Heatmap resumen
print(f"\n📈 GENERANDO VISUALIZACIÓN RESUMEN...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Preparar datos para heatmaps
def crear_matriz_resumen(df_bins, metricas=['f1', 'recall', 'precision', 'error_rms']):
    """Crea matriz para heatmap con métricas por bins de SNR"""
    if len(df_bins) == 0:
        return None, None

    matriz = []
    labels_bins = []

    for _, row in df_bins.iterrows():
        fila = [row[m] for m in metricas]
        matriz.append(fila)
        labels_bins.append(row['bin_snr'])

    return np.array(matriz), labels_bins

# Heatmap Fase P
matriz_p, labels_p = crear_matriz_resumen(df_bins_p)
if matriz_p is not None:
    im1 = axes[0,0].imshow(matriz_p.T, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)
    axes[0,0].set_title('Métricas Fase P por Bins de SNR')
    axes[0,0].set_xticks(range(len(labels_p)))
    axes[0,0].set_xticklabels(labels_p, rotation=45)
    axes[0,0].set_yticks(range(4))
    axes[0,0].set_yticklabels(['F1-Score', 'Recall', 'Precision', 'Error RMS'])

    # Añadir valores en las celdas
    for i in range(len(labels_p)):
        for j in range(4):
            if j < 3:  # Para F1, Recall, Precision
                texto = f'{matriz_p[i,j]:.2f}'
            else:  # Para Error RMS
                texto = f'{matriz_p[i,j]:.1f}s'
            axes[0,0].text(i, j, texto, ha='center', va='center',
                          color='white' if matriz_p[i,j] < 0.5 else 'black', fontsize=9)

# Heatmap Fase S
matriz_s, labels_s = crear_matriz_resumen(df_bins_s)
if matriz_s is not None:
    im2 = axes[0,1].imshow(matriz_s.T, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)
    axes[0,1].set_title('Métricas Fase S por Bins de SNR')
    axes[0,1].set_xticks(range(len(labels_s)))
    axes[0,1].set_xticklabels(labels_s, rotation=45)
    axes[0,1].set_yticks(range(4))
    axes[0,1].set_yticklabels(['F1-Score', 'Recall', 'Precision', 'Error RMS'])

    for i in range(len(labels_s)):
        for j in range(4):
            if j < 3:
                texto = f'{matriz_s[i,j]:.2f}'
            else:
                texto = f'{matriz_s[i,j]:.1f}s'
            axes[0,1].text(i, j, texto, ha='center', va='center',
                          color='white' if matriz_s[i,j] < 0.5 else 'black', fontsize=9)

# Scatter combinado: SNR vs F1 para ambas fases
if len(df_bins_p) > 0 and len(df_bins_s) > 0:
    axes[1,0].scatter(df_bins_p['snr_mean'], df_bins_p['f1'],
                     label='Fase P', alpha=0.8, s=100, color='blue')
    axes[1,0].scatter(df_bins_s['snr_mean'], df_bins_s['f1'],
                     label='Fase S', alpha=0.8, s=100, color='red')

    axes[1,0].set_xlabel('SNR Medio (dB)')
    axes[1,0].set_ylabel('F1-Score')
    axes[1,0].set_title('F1-Score vs SNR (Ambas Fases)')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)

    # Líneas de tendencia
    if len(df_bins_p) > 2:
        z_p = np.polyfit(df_bins_p['snr_mean'], df_bins_p['f1'], 1)
        p_p = np.poly1d(z_p)
        x_trend = np.linspace(df_bins_p['snr_mean'].min(), df_bins_p['snr_mean'].max(), 100)
        axes[1,0].plot(x_trend, p_p(x_trend), "b--", alpha=0.8, linewidth=2)

    if len(df_bins_s) > 2:
        z_s = np.polyfit(df_bins_s['snr_mean'], df_bins_s['f1'], 1)
        p_s = np.poly1d(z_s)
        x_trend_s = np.linspace(df_bins_s['snr_mean'].min(), df_bins_s['snr_mean'].max(), 100)
        axes[1,0].plot(x_trend_s, p_s(x_trend_s), "r--", alpha=0.8, linewidth=2)

# Distribución de eventos por bins de SNR
if len(df_bins_p) > 0:
    axes[1,1].bar(range(len(df_bins_p)), df_bins_p['n_eventos'],
                 alpha=0.7, color='skyblue', label='Fase P')
    axes[1,1].set_xlabel('Bins de SNR-P')
    axes[1,1].set_ylabel('Número de Eventos')
    axes[1,1].set_title('Distribución de Eventos por Bins de SNR')
    axes[1,1].set_xticks(range(len(df_bins_p)))
    axes[1,1].set_xticklabels([f'{b}' for b in df_bins_p['bin_snr']], rotation=45)
    axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Añadir colorbar
fig.colorbar(im1 if matriz_p is not None else im2, ax=axes[0,:],
             label='Valor de Métrica', shrink=0.6)

In [ ]:
# Análisis predictivo: ¿Qué SNR necesito para obtener X performance?
print(f"\n🔮 ANÁLISIS PREDICTIVO:")
print(f"¿Qué SNR necesito para obtener cierta performance?")

def predecir_snr_necesario(df_bins, metrica_objetivo, valor_objetivo):
    """Predice el SNR necesario para alcanzar un valor objetivo de métrica"""

    if len(df_bins) < 2:
        return None

    # Ordenar por SNR
    df_sorted = df_bins.sort_values('snr_mean')

    # Interpolación lineal simple
    snr_vals = df_sorted['snr_mean'].values
    metrica_vals = df_sorted[metrica_objetivo].values

    # Encontrar puntos para interpolación
    indices_validos = ~np.isnan(metrica_vals)
    if np.sum(indices_validos) < 2:
        return None

    snr_clean = snr_vals[indices_validos]
    metrica_clean = metrica_vals[indices_validos]

    # Interpolación
    try:
        snr_predicho = np.interp(valor_objetivo, metrica_clean, snr_clean)
        return snr_predicho
    except:
        return None

# Objetivos de performance típicos
objetivos_performance = [
    ('F1-Score = 0.8', 'f1', 0.8),
    ('F1-Score = 0.7', 'f1', 0.7),
    ('Recall = 0.9', 'recall', 0.9),
    ('Recall = 0.8', 'recall', 0.8),
    ('Precision = 0.8', 'precision', 0.8),
    ('Error RMS = 1.5s', 'error_rms', 1.5)
]

print(f"\nFASE P:")
for objetivo_desc, metrica, valor in objetivos_performance:
    if metrica == 'error_rms':
        # Para error RMS, necesitamos invertir la lógica
        df_invertido = df_bins_p.copy()
        df_invertido['error_rms_inv'] = 1.0 / (df_invertido['error_rms'] + 0.1)  # Evitar división por 0
        snr_pred = predecir_snr_necesario(df_invertido, 'error_rms_inv', 1.0/(valor + 0.1))
    else:
        snr_pred = predecir_snr_necesario(df_bins_p, metrica, valor)

    if snr_pred:
        print(f"  • Para {objetivo_desc}: SNR-P ≈ {snr_pred:.1f} dB")
    else:
        print(f"  • Para {objetivo_desc}: No se puede predecir (datos insuficientes)")

print(f"\nFASE S:")
for objetivo_desc, metrica, valor in objetivos_performance:
    if metrica == 'error_rms':
        df_invertido = df_bins_s.copy()
        df_invertido['error_rms_inv'] = 1.0 / (df_invertido['error_rms'] + 0.1)
        snr_pred = predecir_snr_necesario(df_invertido, 'error_rms_inv', 1.0/(valor + 0.1))
    else:
        snr_pred = predecir_snr_necesario(df_bins_s, metrica, valor)

    if snr_pred:
        print(f"  • Para {objetivo_desc}: SNR-S ≈ {snr_pred:.1f} dB")
    else:
        print(f"  • Para {objetivo_desc}: No se puede predecir (datos insuficientes)")

# Validación de hipótesis final
print(f"\n" + "="*80)
print("VALIDACIÓN DE HIPÓTESIS")
print("="*80)

hipotesis_original = "Un bajo SNR dificulta la detección por parte del modelo GPD"

print(f"💡 HIPÓTESIS ORIGINAL: {hipotesis_original}")
print(f"\n✅ VALIDACIÓN:")

# Evidencias recolectadas
evidencias = []

# 1. Correlación SNR vs métricas
if len(df_correlaciones) > 0:
    corr_significativas = df_correlaciones[df_correlaciones['Significativa'] == 'Sí']
    if len(corr_significativas) > 0:
        evidencias.append(f"Correlaciones significativas encontradas entre SNR y {len(corr_significativas)} métricas")

# 2. Variación en bins
if len(df_bins_p) > 1:
    var_f1_p = df_bins_p['f1'].max() - df_bins_p['f1'].min()
    if var_f1_p > 0.2:
        evidencias.append(f"F1-Score Fase P varía {var_f1_p:.3f} puntos entre bins de SNR")

if len(df_bins_s) > 1:
    var_f1_s = df_bins_s['f1'].max() - df_bins_s['f1'].min()
    if var_f1_s > 0.2:
        evidencias.append(f"F1-Score Fase S varía {var_f1_s:.3f} puntos entre bins de SNR")

# 3. Tendencias monotónicas
if len(df_bins_p) > 2:
    # Verificar si F1 tiende a aumentar con SNR
    corr_f1_snr_p = df_bins_p['snr_mean'].corr(df_bins_p['f1'])
    if corr_f1_snr_p > 0.5:
        evidencias.append(f"Tendencia positiva fuerte SNR-P vs F1-Score (r={corr_f1_snr_p:.3f})")

if len(df_bins_s) > 2:
    corr_f1_snr_s = df_bins_s['snr_mean'].corr(df_bins_s['f1'])
    if corr_f1_snr_s > 0.5:
        evidencias.append(f"Tendencia positiva fuerte SNR-S vs F1-Score (r={corr_f1_snr_s:.3f})")

# Imprimir evidencias
for i, evidencia in enumerate(evidencias, 1):
    print(f"  {i}. {evidencia}")

# Conclusión final
if len(evidencias) >= 2:
    print(f"\n🎯 CONCLUSIÓN: HIPÓTESIS CONFIRMADA")
    print(f"   La evidencia cuantitativa respalda fuertemente que el SNR bajo")
    print(f"   impacta negativamente el rendimiento del modelo GPD.")
else:
    print(f"\n⚠️  CONCLUSIÓN: EVIDENCIA LIMITADA")
    print(f"   Se requieren más datos para confirmar completamente la hipótesis.")

print(f"\n💾 TODOS LOS ANÁLISIS GUARDADOS EN:")
print(f"   • /content/analisis_snr_bins_fase_P.csv")
print(f"   • /content/analisis_snr_bins_fase_S.csv")
print(f"   • /content/correlaciones_snr_metricas.csv")
print(f"   • /content/sesgos_temporales_snr_P.csv")
print(f"   • /content/sesgos_temporales_snr_S.csv")

print(f"\n" + "="*80)
print("ANÁLISIS COMPLETADO")
print("="*80)

Errores en las detecciones de las fases para cada estación

In [ ]:
# Cargar los datos de análisis completo
print("=== MÉTRICAS POR ESTACIÓN: SNR Y ERRORES DEL MODELO GPD ===")

# Cargar el archivo con el análisis completo del impacto del modelo
df_impacto = pd.read_csv('/content/analisis_completo_impacto_modelo.csv')
df_robustez = pd.read_csv('/content/analisis_robustez_estaciones.csv')

# Combinar información relevante
df_resumen = pd.merge(
    df_impacto[['Estacion', 'N_eventos', 'error_p_mean', 'error_p_median', 'error_p_rms',
                'error_s_mean', 'error_s_median', 'error_s_rms', 'Clasificacion']],
    df_robustez[['Estacion', 'snr_p_mean', 'snr_p_median', 'snr_s_mean', 'snr_s_median']],
    on='Estacion'
)

print("✓ Datos cargados y combinados")
print(f"✓ Estaciones analizadas: {len(df_resumen)}")

In [ ]:
# Tabla resumen completa por estación
print("\n" + "="*120)
print("RESUMEN COMPLETO POR ESTACIÓN")
print("="*120)

# Ordenar por clasificación (Problemática -> Intermedia -> Robusta)
orden_clasificacion = {'Problemática': 1, 'Intermedia': 2, 'Robusta': 3}
df_resumen['orden'] = df_resumen['Clasificacion'].map(orden_clasificacion)
df_resumen_ordenado = df_resumen.sort_values('orden')

# Mostrar tabla completa
print(f"{'Estación':<8} {'Clasif':<12} {'N_Evt':<6} {'SNR-P':<15} {'SNR-S':<15} {'Error-P (s)':<25} {'Error-S (s)':<25}")
print(f"{'':<8} {'':<12} {'':<6} {'Med|Mean':<15} {'Med|Mean':<15} {'Med|Mean|RMS':<25} {'Med|Mean|RMS':<25}")
print("-" * 120)

for _, row in df_resumen_ordenado.iterrows():
    estacion = row['Estacion']
    clasificacion = row['Clasificacion']
    n_eventos = int(row['N_eventos'])

    # SNR P
    snr_p_med = row['snr_p_median']
    snr_p_mean = row['snr_p_mean']
    snr_p_str = f"{snr_p_med:.1f}|{snr_p_mean:.1f}"

    # SNR S
    snr_s_med = row['snr_s_median']
    snr_s_mean = row['snr_s_mean']
    snr_s_str = f"{snr_s_med:.1f}|{snr_s_mean:.1f}"

    # Errores P
    if pd.notna(row['error_p_median']) and pd.notna(row['error_p_mean']) and pd.notna(row['error_p_rms']):
        error_p_str = f"{row['error_p_median']:+.2f}|{row['error_p_mean']:+.2f}|{row['error_p_rms']:.2f}"
    else:
        error_p_str = "Sin datos suficientes"

    # Errores S
    if pd.notna(row['error_s_median']) and pd.notna(row['error_s_mean']) and pd.notna(row['error_s_rms']):
        error_s_str = f"{row['error_s_median']:+.2f}|{row['error_s_mean']:+.2f}|{row['error_s_rms']:.2f}"
    else:
        error_s_str = "Sin datos suficientes"

    print(f"{estacion:<8} {clasificacion:<12} {n_eventos:<6} {snr_p_str:<15} {snr_s_str:<15} {error_p_str:<25} {error_s_str:<25}")

print("-" * 120)
print("Nota: Errores con signo + indican que el modelo detecta TARDE, - indica detección TEMPRANA")
print("      RMS siempre es positivo y mide la magnitud total del error")